## Exercice 1

In [1]:
!pip3 install urllib3
!pip3 install pandas
!pip3 install numpy

In [2]:
# Retrieve prediction from the wiidii model hosted on docker webservice 
import urllib.request
# Structure of persistant data
import json 

# Manipulation of tabular data
import pandas as pd
# Calculations with numericals
import numpy as np

In [3]:
#Reading training_set file 
with open('training_set.json', encoding="utf8") as json_file:
    training_data = json.load(json_file)
    
#Reading testing_set file 
with open('testing_set.json', encoding="utf8") as json_file:
    testing_data = json.load(json_file)

In [4]:
# Parsing data from json structure to pandas Dataframe structure
dataset_train = np.array([sample["sentence"] for sample in training_data])
target_train  = np.array([sample["intent"] for sample in training_data])

dataset_test = np.array([sample["sentence"] for sample in testing_data])
target_test  = np.array([sample["intent"] for sample in testing_data])

In [5]:
# Checking shapes
print(f"Dataset shape : {dataset_train.shape}");
print(f"Target  shape : {target_train.shape}\n");

print(f"Dataset shape : {dataset_test.shape}");
print(f"Target  shape : {target_test.shape}");

Dataset shape : (6035,)
Target  shape : (6035,)

Dataset shape : (1065,)
Target  shape : (1065,)


In [10]:
# Changing types for better visualisation & computations
x_train = pd.DataFrame(dataset_train,columns=["Sentence"])
y_train = pd.DataFrame(target_train, columns=["Intent"])

x_test = pd.DataFrame(dataset_test,columns=["Sentence"])
y_test = pd.DataFrame(target_test, columns=["Intent"])

In [12]:
def study_data(x):
    # Studying input data
    print(x.head())
    print("\n")
    # Missing value
    print(x.info())
    print("\n")
    # x.describe() no need for descriptive stats because categorical object

study_data(x_test)

                                            Sentence
0            Tu sais où je peux acheter un bazooka ?
1                 déco de maison style maison ferret
2                      j&#39;ai envie qu&#39;on se rencontre
3             J’ai besoin de fleurs pr ma grand mère
4  trouve à champigny-sur-marne où manger de la c...


&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
RangeIndex: 1065 entries, 0 to 1064
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  1065 non-null   object
dtypes: object(1)
memory usage: 8.4+ KB
None




In [13]:
import urllib.parse
# Getting prediction with 1065 test data
def get_prediction_from_docker(sentence):
    url = "http://127.0.0.1:80/api/intent?sentence="+"<"+urllib.parse.quote(sentence)+">"
    with urllib.request.urlopen(url) as response :
        return json.load(response)

In [15]:
json_results = [get_prediction_from_docker(sentence) for sentence in x_test['Sentence']]

In [21]:
json_results[0]

{&#39;find-train&#39;: 0.020727816969156265,
 &#39;irrelevant&#39;: 0.3929397463798523,
 &#39;find-flight&#39;: 0.03598225861787796,
 &#39;find-restaurant&#39;: 0.07283203303813934,
 &#39;purchase&#39;: 0.37030187249183655,
 &#39;find-around-me&#39;: 0.06633546948432922,
 &#39;provide-showtimes&#39;: 0.009604641236364841,
 &#39;find-hotel&#39;: 0.03127620369195938}

In [57]:
# Prepare to add the prediction feature in our final dataframe
prediction_columns = ['find-train', 'irrelevant', 'find-flight', 'find-restaurant',
           'purchase', 'find-around-me', 'provide-showtimes', 'find-hotel']
predictions = [prediction_columns[np.argmax([json_results[i][value] for value in json_results[i]])] for i in range(len(json_results))]
print(predictions[:5])

[&#39;irrelevant&#39;, &#39;irrelevant&#39;, &#39;irrelevant&#39;, &#39;irrelevant&#39;, &#39;find-restaurant&#39;]


In [58]:

df_results = pd.DataFrame(json_results, columns=prediction_columns)
df_prediction = pd.DataFrame(predictions, columns=["Predicted"])

In [59]:
print(f"Nb de resultats : {len(df_results)}")
df_results.head()

Nb de resultats : 1065


,find-train,irrelevant,find-flight,find-restaurant,purchase,find-around-me,provide-showtimes,find-hotel
0,0.020728,0.392940,0.035982,0.072832,0.370302,0.066335,0.009605,0.031276
1,0.034970,0.531172,0.035429,0.038700,0.080105,0.054041,0.029561,0.196022
2,0.017667,0.681269,0.025598,0.134089,0.050936,0.029113,0.021443,0.039883
3,0.021682,0.429878,0.031576,0.038159,0.267280,0.117537,0.014895,0.078994
4,0.027025,0.154664,0.010786,0.657717,0.098362,0.023528,0.015831,0.012087


In [60]:
print(f"Nb de resultats : {len(df_prediction)}")
df_prediction.head()

Nb de resultats : 1065


,Predicted
0,irrelevant
1,irrelevant
2,irrelevant
3,irrelevant
4,find-restaurant


In [61]:
print(f"Nb d'input : {len(x_test)}")
x_test.head()

Nb d&#39;input : 1065


,Sentence
0,Tu sais où je peux acheter un bazooka ?
1,déco de maison style maison ferret
2,j'ai envie qu'on se rencontre
3,J’ai besoin de fleurs pr ma grand mère
4,trouve à champigny-sur-marne où manger de la c...


In [28]:
# Merge input and prediction for further visualisation
df_final = df_results.copy()
df_final["Sentence"]  = x_test["Sentence"]
df_final["Predicted"] = df_prediction["Predicted"]
df_final["Intent"]    = y_test["Intent"]
df_final.head()

,find-train,irrelevant,find-flight,find-restaurant,purchase,find-around-me,provide-showtimes,find-hotel,Sentence,Intent
0,0.020728,0.392940,0.035982,0.072832,0.370302,0.066335,0.009605,0.031276,Tu sais où je peux acheter un bazooka ?,irrelevant
1,0.034970,0.531172,0.035429,0.038700,0.080105,0.054041,0.029561,0.196022,déco de maison style maison ferret,irrelevant
2,0.017667,0.681269,0.025598,0.134089,0.050936,0.029113,0.021443,0.039883,j'ai envie qu'on se rencontre,irrelevant
3,0.021682,0.429878,0.031576,0.038159,0.267280,0.117537,0.014895,0.078994,J’ai besoin de fleurs pr ma grand mère,purchase
4,0.027025,0.154664,0.010786,0.657717,0.098362,0.023528,0.015831,0.012087,trouve à champigny-sur-marne où manger de la c...,find-restaurant


In [ ]:
df_final.to_csv()

In [61]:
#### PARTIE HIBA, NE PAS TOUCHER

#Counting the number of data (train) for each intent
train_data = {'find-train': 0, 'irrelevant': 0, 'find-flight': 0, 'find-restaurant': 0,
           'purchase': 0, 'find-around-me' : 0, 'provide-showtimes' : 0, 'find-hotel' : 0}
for i in range(len(training_data)):
    for intent, count in train_data.items():
        if training_data[i]['intent'] == intent :
            count = count + 1
            train_data[intent] = count

#Counting each intent iteration
train_data

{'find-train': 143,
 'irrelevant': 3852,
 'find-flight': 142,
 'find-restaurant': 469,
 'purchase': 613,
 'find-around-me': 383,
 'provide-showtimes': 117,
 'find-hotel': 316}

In [63]:
#Counting the number of data (test) for each intent
test_data = {'find-train': 0, 'irrelevant': 0, 'find-flight': 0, 'find-restaurant': 0,
           'purchase': 0, 'find-around-me' : 0, 'provide-showtimes' : 0, 'find-hotel' : 0}
for i in range(len(testing_data)):
    for intent, count in test_data.items():
        if testing_data[i]['intent'] == intent :
            count = count + 1
            test_data[intent] = count

#Counting each intent iteration
test_data

{'find-train': 21,
 'irrelevant': 677,
 'find-flight': 24,
 'find-restaurant': 93,
 'purchase': 114,
 'find-around-me': 67,
 'provide-showtimes': 14,
 'find-hotel': 55}

## Exercice 2

##### Réponse / idées : 

## Exercice 3

##### Réponse / idées : 

## Exercice 4

##### Réponse / idées : 

## Exercice 5

##### Réponse / idées : 